In [1]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

c:\Users\hp\Desktop\DS2\Natural Language Processing\project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NER Model loading...

In [2]:
NER_model = AutoModelForTokenClassification.from_pretrained("./saved_model")
print(NER_model.config.problem_type)
NER_model.config.problem_type = 'token_classification'
print(NER_model.config.problem_type)
NER_tokenizer = AutoTokenizer.from_pretrained("./saved_model")
ner_pipeline = pipeline("ner", model=NER_model, tokenizer=NER_tokenizer, aggregation_strategy="simple") 

Device set to use cpu


multi_label_classification
token_classification


(Existing) FinSentiment Analysis Model loading...

In [3]:
sent_tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
sent_model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
sent_pipeline = pipeline("sentiment-analysis", model=sent_model, tokenizer=sent_tokenizer)

Device set to use cpu


Test on trump tweets

In [6]:
tweets = pd.read_csv("test_data/proc_data.csv")
tweets['Tweet Text'] = tweets['Tweet Text'].str[2:-1]
tweet = tweets['Tweet Text']

entities_list = []
sent_list = []
k = 10
for sentence in tweet[100:112]:
    entities = ner_pipeline(sentence)
    sent = sent_pipeline(sentence)
    extracted_entities = [{"entity": ent["entity_group"], "word": ent["word"], "start": ent["start"], "end": ent["end"]} for ent in entities]
    entities_list.append(extracted_entities)
    sent_list.append(sent)

# Print results
for i, (sent, ents, s) in enumerate(zip(tweet[100:112], entities_list, sent_list)):
    print(f"Sentence {i+1}: {sent}")
    print("Entities:", ents)
    print("Sentiment:", s)
    print()

Sentence 1: Enjoy the # SuperBowl and then we continue: MAKE AMERICA GREAT AGAIN!
Entities: [{'entity': 'LOC_B', 'word': 'america', 'start': 49, 'end': 56}]
Sentiment: [{'label': 'neutral', 'score': 0.9960447549819946}]

Sentence 2: What an amazing comeback and win by the Patriots. Tom Brady Bob Kraft and Coach B are total winners. Wow!
Entities: [{'entity': 'ORG_B', 'word': 'patriot', 'start': 40, 'end': 47}, {'entity': 'PER_B', 'word': 'tom', 'start': 50, 'end': 53}, {'entity': 'PER_I', 'word': 'brady', 'start': 54, 'end': 59}, {'entity': 'PER_B', 'word': 'bob', 'start': 60, 'end': 63}, {'entity': 'PER_I', 'word': 'kraft', 'start': 64, 'end': 69}, {'entity': 'ORG_B', 'word': 'coach', 'start': 74, 'end': 79}, {'entity': 'ORG_I', 'word': 'b', 'start': 80, 'end': 81}]
Sentiment: [{'label': 'positive', 'score': 0.9985207915306091}]

Sentence 3: Any negative polls are fake news just like the CNN ABC NBC polls in the election. Sorry people want border security and extreme vetting.
Entities